In [8]:
import visa
import time as t
from copy import deepcopy
import numpy as np
from math import pi, sin
from qiskit import *
import sys

In [9]:
rm = visa.ResourceManager()
inst0 = rm.open_resource('USB0::0x05E6::0x2450::04399313::INSTR')
inst1 = rm.open_resource('USB0::0x05E6::0x2450::04399315::INSTR')
inst2 = rm.open_resource('USB0::0x05E6::0x2450::04434061::INSTR')

In [16]:
delay = 0.005
class BlochSphere:
    rm = visa.ResourceManager()
    inst0 = rm.open_resource('USB0::0x05E6::0x2450::04399313::INSTR')
    inst1 = rm.open_resource('USB0::0x05E6::0x2450::04399315::INSTR')
    inst2 = rm.open_resource('USB0::0x05E6::0x2450::04434061::INSTR')
    
    def __init__(self,circ,q,c):
        self._circ = circ
        self._q = q
        self._c = c
        #self.rm = rm = visa.ResourceManager()
        
    def getCounts(self,circ,q,c,shots):
        job = execute(circ, backend = Aer.get_backend('qasm_simulator'), shots = shots)
        counts = job.result().get_counts(circ)
        return counts
        
    def x_comp(self,circ0,q,c):
        delay = 0.1
        circ0 = self._circ
        q = self._q
        c = self._c
        shots = 10000
        
        circ = deepcopy(circ0)
        circ.ry(-pi/2,q)
        circ.measure(q,c)
        c = self.getCounts(circ,q,c,shots)
        x = (c.get('0',0) - c.get('1',0))/shots
        print(x)
        
        inst0.write(':OUTP ON')
        for i in range(abs(round(x*100))):
            inst0.write('SOUR:CURR ' + str((i/100) * (abs(x)/x)))
            t.sleep(delay)
        
    def y_comp(self,circ0,q,c):
        delay = 0.1
        circ0 = self._circ
        q = self._q
        c = self._c
        shots = 10000
        
        circ = deepcopy(circ0)
        circ.rx(pi/2,q)
        circ.measure(q,c)
        c = self.getCounts(circ,q,c,shots)
        y = (c.get('0',0) - c.get('1',0))/shots
        print(y)
        inst1.write(':OUTP ON')
        for i in range(abs(round(y*100))):
            inst1.write('SOUR:CURR ' + str((i/100) * (abs(y)/y)))
            t.sleep(delay)
        
    def z_comp(self,circ0,q,c):
        delay = 0.1
        circ0 = self._circ
        q = self._q
        c = self._c
        shots = 10000
        
        circ = deepcopy(circ0)
        circ.measure(q,c)
        c = self.getCounts(circ,q,c,shots)
        z = (c.get('0',0) - c.get('1',0))/shots
        print(z)
        
        inst2.write(':OUTP ON')
        for i in range(abs(round(z*100))):
            inst2.write('SOUR:CURR ' + str((i/100) * (abs(z)/z)))
            t.sleep(delay)
    def off(self):
        inst0.write(':OUTP OFF')
        inst1.write(':OUTP OFF')
        inst2.write(':OUTP OFF')
    def run(self,circ,q,c):
        self.x_comp(circ,q,c)
        self.y_comp(circ,q,c)
        self.z_comp(circ,q,c)
  

In [17]:
def initialize():
    inst2.write(':OUTP ON')
    inst2.write('SOUR:CURR 1.0')

In [25]:
q = QuantumRegister(1)
c = ClassicalRegister(1)
circ1 = QuantumCircuit(q,c)
circ1.h(q)
circ1.rz(pi/2,q)
initialize()


In [23]:
b1 = BlochSphere(circ1, q,c)
b1.run(circ1,q,c)

0.0006
1.0
0.0004


In [24]:
b1.off()